## Импорт всех библиотек

In [1]:
import pandas as pd
import sklearn
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import gc
from sklearn import linear_model
%matplotlib inline
import numpy as np
import warnings 
from scipy import stats, integrate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score, auc, precision_recall_curve, roc_curve, 
    precision_score, recall_score)
from xgboost import XGBClassifier
import warnings
from sklearn import preprocessing
warnings.simplefilter("ignore")
from sklearn import metrics


/opt/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Чтение Файла

In [2]:
data = pd.read_csv('churn_df.df')

### Переиминовывание колонок

In [3]:
data = data.rename(columns = {
    'success_mobile_login':'success_login',
    'confirmed_mobile_eoperations':'confirmed_eoperations',
    'rejected_mobile_eoperations':'rejected_eoperations',
    'sum_mobile_epay_rur':'sum_epay_rur',
    'count_mobile_epay_rur':'count_epay_rur',
    'sum_mobile_epay_fee':'sum_epay_fee',
    'count_mobile_epay_fee':'count_epay_fee'})


### Чистка данных

In [4]:

%%time
data['week_start'] = pd.to_datetime(data['week_start'])
data = data.loc[data['week_start'] > '2016-01-01']

list_to_0 = ['count_debit','count_credit','sum_card_trans_rur','count_card_trans','num_cardtype','success_login','client_life_span',
            'sum_epay_rur','count_epay_rur','sum_epay_fee','count_epay_fee','confirmed_eoperations','rejected_eoperations']

for column in list_to_0:
    data[column]=data[column].fillna(0)

data['gender_uk']=data.groupby('client_pin')['gender_uk'].transform(lambda x: x.bfill().ffill())

data['gender_uk'] = data['gender_uk'].fillna(data['gender_uk'].median())
data['country_uk'] = data['country_uk'].fillna(data['country_uk'].median())

def fill_age(x):
    delta_day = 7/365.2425
    age_add_range = pd.Series(range(len(x)), index=x.index)*delta_day
    first_not_null_idx = x.first_valid_index() - x.index[0]
    to_fill = age_add_range + (x[x.first_valid_index()] - first_not_null_idx*delta_day)
    return x.fillna(to_fill)

CPU times: user 11min 8s, sys: 31 s, total: 11min 39s
Wall time: 11min 29s


In [5]:
%%time
#filling age
q = data.groupby('client_pin')['age'].apply(lambda x: x.sum() != 0)
pins_null = q[~q].index
q1 = data[data.client_pin.isin(pins_null)]
med = data.age.median()
q2 = q1.groupby('client_pin')['age'].apply(lambda x: fill_age(x.fillna(med, limit=1)))

data.loc[data.client_pin.isin(pins_null),'age'] = q2

data['age'] = data.groupby('client_pin')['age'].apply(lambda x: fill_age(x) if x.notnull().sum() != 0 else x.fillna(0))

CPU times: user 37min 10s, sys: 32.4 s, total: 37min 43s
Wall time: 37min 40s


#### Сохранение предобработанной таблицы

In [6]:
%%time
data.to_hdf(path+'/all_clean_fin.h5', 'df', mode='w')

CPU times: user 14.9 s, sys: 39.5 s, total: 54.4 s
Wall time: 4min 9s


In [ ]:
#data= pd.read_hdf(path+'/all_clean_fin.h5', 'df')

### Создание аггрегатов переменных

In [7]:
%%time
data = data.drop('rejected_eoperations',axis=1)
list_rol = [
 'sum_card_trans_rur',
 'count_card_trans',
 'success_login',
 'sum_epay_rur',
 'count_epay_rur',
 'sum_epay_fee',
 'count_epay_fee',
 'confirmed_eoperations']

CPU times: user 3.47 s, sys: 4.83 s, total: 8.3 s
Wall time: 8.23 s


In [ ]:
%%time
for column in list_rol:
    data['rol_4_'+column]=pd.DataFrame(pd.rolling_sum(data.groupby('client_pin')[column], window=4, min_periods=4)).reset_index(drop=True)

for column in list_rol:
    data['rol_12_'+column]=pd.DataFrame(pd.rolling_sum(data.groupby('client_pin')[column], window=12, min_periods=12)).reset_index(drop=True)
data=data.fillna(0)

CPU times: user 4h 12min 17s, sys: 15min 20s, total: 4h 27min 37s
Wall time: 4h 30min 36s


### Сохранение таблицы готовой к моделированию

In [ ]:
%%time
data.to_hdf(path+'/all_feat_fin.h5', 'df', mode='w')

In [81]:
import pandas as pd

am_active = pd.read_csv('/home/alfadata/data/churn/active_clients/am_active_201702.csv',sep=';',header=None)
am_active1 = pd.read_csv('/home/alfadata/data/churn/active_clients/am_active_201703.csv',sep=';',header=None)

bank_active = pd.read_csv('/home/alfadata/data/churn/active_clients/active_client_201702.csv',sep=';')

pins_notA_mobile = am_active[am_active[1]==0][0]
pins_notA_mobile1 = am_active1[am_active1[1]==0][0]
pins_notA_mobile = pd.concat([pins_notA_mobile1,pins_notA_mobile],axis=0)
pins_notA_mobile=pins_notA_mobile.drop_duplicates()

pins_notA_bank = bank_active[bank_active['act_new2015']==0]['pin_eq']

all_Nactive_pins = pd.concat([pins_notA_bank,pins_notA_mobile])

all_Nactive_pins = all_Nactive_pins.drop_duplicates()

data = pd.read_hdf('/home/alfadata/data/churn/cleaned_df/all_feat_fin.h5', 'df')


intersection = set(data['client_pin'].unique()).intersection(all_Nactive_pins)

list(intersection)#все неактивные в data

data=data[~data['client_pin'].isin(intersection)]


#убираение строк где клиент еще не существовал
# data = data[data['client_life_span']>=7]

#убераем те пины которые входили только один раз за все время
q = data.groupby('client_pin')['success_login'].sum()>1
q=q[q]
q = pd.DataFrame(q)
data = data[data['client_pin'].isin(q.index)]

### Разбиение данных на трейн и тест. Создание таргетовой переменной

In [82]:
%%time

week_to_pred = list(range(data.week_start.nunique()))[-1]
num_weeks_analyze = 12

list_weeks = list(range(data.week_start.nunique()-3))
list_weeks_analyze = list_weeks[-num_weeks_analyze:]
df_analyze = data.groupby('client_pin').nth(list_weeks_analyze).reset_index()


pin_analyze = df_analyze.groupby('client_pin')['target'].sum()
pins_not_use = pin_analyze[pin_analyze == 0].index
data = data[~data.client_pin.isin(list(pins_not_use))]



pins_to_pred = data.groupby('client_pin')['age'].sum().index


#энкодинг сегмента
#cols_to_drop = ['crm_category','income_segment','productgroup']
#data = data.drop(cols_to_drop, axis = 1)

#Препроцессинг всех фичей
#data.iloc[:,3:] = data.iloc[:,3:].apply(
 #                          lambda x: preprocessing.StandardScaler().fit_transform(x))


CPU times: user 26.4 s, sys: 10.2 s, total: 36.6 s
Wall time: 36.6 s


In [83]:

#Разделение на трейн и тест на to_pred не происходит обучения. 
#Последняя неделя без учета еще 3х последних(они вообще не участвуют в обучении и предсказании)
to_pred = data.groupby('client_pin').nth(week_to_pred).reset_index()

data['target']=data.groupby('client_pin')['target'].shift(-3).rolling(window=4,min_periods=1).sum().clip_upper(1)#.reset_index()['target']
#test_dec = data.groupby('client_pin').nth([50,51,52,53]).reset_index()

# list_weeks = 
data = data.groupby('client_pin').nth(list_weeks).reset_index()
data = data[data['client_life_span']>=7]


In [80]:
data['target'].value_counts()

1.0    28393156
0.0     7089717
Name: target, dtype: int64

### Моделирование

In [84]:
metrics_list = []
list_weeks = list(range(data.week_start.nunique()))
list_weeks_analyze_train = list_weeks[0:-1]
list_weeks_analyze_test = list_weeks[-1]
train = data.groupby('client_pin').nth(list_weeks_analyze_train).reset_index()
test = data.groupby('client_pin').nth(list_weeks_analyze_test).reset_index()
cols_to_drop = ['target', 'client_pin', 'week_start']

In [87]:
%%time
y_train_Fold = train['target']
X_train_Fold = train.drop(cols_to_drop, axis=1)
X_test_Fold = test.drop(cols_to_drop, axis=1)
y_test_Fold = test['target']
d_train =xgb.DMatrix(X_train_Fold, label=y_train_Fold)
d_valid = xgb.DMatrix(X_test_Fold, label=y_test_Fold)
params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.8,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'nthread':35,
    'booster': 'gbtree',
    'eval_metric':'auc'}

watchlist = [(d_train, 'train'), (d_valid, 'eval')]
# if i == 5:
#     verbose_eval = True
# else:
#     verbose_eval = False
clf = xgb.train(params,
                d_train,
                100000,
                watchlist,
                early_stopping_rounds=100,verbose_eval=10
                )

pred = clf.predict(d_valid, ntree_limit=clf.best_ntree_limit)
#pred_prob = clf.predict_proba(X_test_Fold)[:, 0]
pred1 = np.round(pred)
auc = metrics.roc_auc_score(y_test_Fold,pred)
recall = metrics.recall_score(pred1,y_test_Fold)
precision = metrics.precision_score(pred1,y_test_Fold)
# metrics_list.append(i)
#     metrics_list.append('recall on fold ' + str(i) + ': ', recall)
#     metrics_list.append('precision on fold ' + str(i) + ': ', precision)
#     metrics_list.append ('AUC on fold ' + str(i) + ': ', auc)
# metrics_list.append(recall)
# metrics_list.append(precision)
# metrics_list.append (auc)

[0]	train-auc:0.900725	eval-auc:0.831629
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 100 rounds.
[10]	train-auc:0.902711	eval-auc:0.835757
[20]	train-auc:0.904355	eval-auc:0.839431
[30]	train-auc:0.905324	eval-auc:0.839924
[40]	train-auc:0.905878	eval-auc:0.840027
[50]	train-auc:0.906285	eval-auc:0.840503
[60]	train-auc:0.906749	eval-auc:0.840974
[70]	train-auc:0.907169	eval-auc:0.840841
[80]	train-auc:0.907451	eval-auc:0.840824
[90]	train-auc:0.907703	eval-auc:0.840894
[100]	train-auc:0.907885	eval-auc:0.84061
[110]	train-auc:0.908018	eval-auc:0.840603
[120]	train-auc:0.908126	eval-auc:0.840465
[130]	train-auc:0.908244	eval-auc:0.840543
[140]	train-auc:0.908311	eval-auc:0.840519
[150]	train-auc:0.908373	eval-auc:0.84038
[160]	train-auc:0.908438	eval-auc:0.840488
Stopping. Best iteration:
[67]	train-auc:0.907008	eval-auc:0.841208

CPU times: user 8h 59min 23s, sys: 28min 43s, total: 9h 28min 7s
Wall t

In [99]:
auc

0.84120844643129533

In [100]:
recall

0.88733216459122455

In [101]:
precision

0.99391554976449514

### Предсказание модели

In [90]:
to_pred = to_pred.drop(cols_to_drop, axis=1)
to_pred = xgb.DMatrix(to_pred)

pred_prob = clf.predict(to_pred)

pred_prob = pd.DataFrame(pred_prob)

pins_to_pred = pd.DataFrame(pins_to_pred)
frames = [pins_to_pred,pred_prob]

df = pd.concat(frames,axis=1)
df = df.rename(columns= {0:'proba'})
df['proba']=df['proba'].apply(lambda x: abs(x-1))
df = df.sort('proba',ascending=False)

In [98]:
df

,client_pin,proba
68289,ABFYPG,0.916823
65976,ABE97K,0.914768
81775,ABREB3,0.914624
70964,ABI120,0.913454
83783,ABTGCJ,0.909915
87554,ABWIHW,0.906604
85194,ABUND1,0.906251
73658,ABK6F5,0.905632
78839,ABOQF2,0.903318
65626,ABE1IU,0.902597


### Сохранение итоговых данных

In [93]:
df.to_csv(path+'/predictions_all.csv',index=False)